In [ ]:
import pandas as pd

import lifelines
from lifelines.utils import to_long_format
from lifelines.utils import add_covariate_to_timeline
from lifelines import CoxTimeVaryingFitter

from scipy.stats import wilcoxon, friedmanchisquare, kruskal

import numpy as np

from matplotlib import pyplot as plt

import seaborn as sns

sns.set(rc={"figure.figsize": (5, 5)})
sns.set_style("ticks")

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
boxplot_props = {
    "boxprops": {"facecolor": "none", "edgecolor": (0.25, 0.25, 0.25)},
    "medianprops": {"color": (0.25, 0.25, 0.25)},
    "whiskerprops": {"color": (0.25, 0.25, 0.25)},
    "capprops": {"color": (0.25, 0.25, 0.25)},
}

# UTSW retrospective cohort

In [ ]:
emboli_data = pd.read_excel("data/Supplementary Table 3.xlsx")

In [ ]:
emboli_data["Medium_Large_Art"] = emboli_data["Medium_Art"] + emboli_data["Large_Art"]
emboli_data["Medium_Large_Vein"] = (
    emboli_data["Medium_Vein"] + emboli_data["Large_Vein"]
)

emboli_data = emboli_data.drop(
    columns=["Medium_Art", "Large_Art", "Medium_Vein", "Large_Vein"]
)

In [ ]:
pt_emboli_data = emboli_data.groupby(["Patient_ID"]).sum().reset_index()

In [ ]:
vals = []
labs = []
hues = []

vars_ = [
    "Medium_Large_Art",
    "Small_Art",
    "Medium_Large_Vein",
    "Small_Vein",
]
var_labels = [
    "Medium/Large Artery",
    "Small Artery",
    "Medium/Large Vein",
    "Small Vein",
]
hue_categories = ["Artery", "Artery", "Vein", "Vein"]

for row in pt_emboli_data.iterrows():
    for var, var_label, hue_cat in zip(vars_, var_labels, hue_categories):
        vals.append(row[1][var])
        labs.append(var_label)
        hues.append(hue_cat)

vals = np.array(vals)

In [ ]:
size_classes = ["Medium_Large", "Small", "Medium_Large", "Small"]
vessel_classes = ["Artery", "Artery", "Vein", "Vein"]

all_pts = list(set(emboli_data["Patient_ID"]))

dists = [[0, 0, 0, 0] for _pt in all_pts]

for row in emboli_data.iterrows():
    this_pt = row[1]["Patient_ID"]
    dists_idx = all_pts.index(this_pt)

    for idx, var in enumerate(vars_):
        if not pd.isnull(row[1][var]):
            dists[dists_idx][idx] += row[1][var]

dists = np.array(dists).T

friedman_res = friedmanchisquare(*dists)

In [ ]:
pairwise_wilcoxon_results = []

for idx_0 in range(len(vars_) - 1):
    var_0 = vars_[idx_0]

    for idx_1 in range(idx_0 + 1, len(vars)):
        var_1 = vars_[idx_1]

        delta = dists[idx_1] - dists[idx_0]
        n_pts_w_increase = len([it for it in delta if it > 0])

        res = wilcoxon(delta, alternative="two-sided")
        pairwise_wilcoxon_results.append(
            (
                var_0,
                var_1,
                res.pvalue,
                np.median(delta),
                np.mean(delta),
                n_pts_w_increase,
            )
        )

In [ ]:
vals += 1
vals = np.log10(vals)

In [ ]:
p = sns.boxplot(
    x=labs, y=vals, saturation=0.8, color="white", fliersize=0, **boxplot_props
)
b = sns.stripplot(
    x=labs,
    y=vals,
    alpha=0.7,
    hue=hues,
    size=6,
)

_ = plt.ylabel("Number of tumor emboli\n(log(count + 1))")


############## anns
ylim = plt.ylim()

stat_annotations_y_start = ylim[0] + (ylim[1] - ylim[0]) / 2.5
stat_annotations_y_start *= 1.4
height = (ylim[1] - ylim[0]) / 6

height_adj_idx = 0

for (
    var_0,
    var_1,
    pval,
    _median_delta,
    _mean_delta,
    _n_pts_w_increase,
) in pairwise_wilcoxon_results:
    if pval < 0.0005:
        pval = f"p={pval:.2e}"
    else:
        pval = f"p={pval:.3f}"
    x1, x2 = vars_.index(var_0), vars_.index(var_1)

    y = stat_annotations_y_start + height * height_adj_idx
    h = (ylim[1] - ylim[0]) / 30

    plt.plot([x1, x1, x2, x2], [y + h, y + (2 * h), y + (2 * h), y + h], lw=1.5, c="k")
    plt.text((x1 + x2) * 0.5, y + (2.5 * h), pval, ha="center", va="bottom", color="k")

    height_adj_idx += 1

#########################

# set y tick labels
_ = plt.yticks(
    [0, 1, 2, 3],
)

_ = p.set_ylim(-0.1, 3)
_ = p.set_yticklabels(["$10^0$", "$10^1$", "$10^2$", "$10^3$"])

xticklabels = p.get_xticklabels()
xticklabels = [" ".join(it.get_text().split()[:-1]) for it in xticklabels]

_ = p.set_xticks(np.arange(len(xticklabels)))
_ = p.set_xticklabels(xticklabels, rotation=45, ha="right")

_ = p.spines["top"].set_visible(False)
_ = p.spines["right"].set_visible(False)

_ = plt.title(
    f"Overall: Friedman test p={friedman_res.pvalue:.2e}\n"
    f"Annotations are for Wilcoxon pairwise comparisons\n"
)

# UTSW prospective cohort

In [ ]:
utsw_prospective_df = pd.read_excel("data/Supplementary Table 7.xlsx")

In [ ]:
premortem_data = {}
postmortem_data = {}

for _idx, row in utsw_prospective_df.iterrows():
    if row["Cohort"] == "Control":
        continue

    pt = row["Patient"]
    visit = row["Visit"]

    if visit == "Post-Mortem":
        postmortem_data[pt] = {
            "Total CTCs": row["Total FITC+ CTCs (count/ml)"],
            "CTC cluster size": row["Mean CTC cluster size: overall"],
            "Total CTC clusters": row["Total FITC+ CTC clusters (count/ml)"],
            "Single CTCs": row["Single FITC+ CTCs (count/ml)"],
            "Homotypic clusters": row["Homotypic FITC+ CTC clusters (count/ml)"],
            "Heterotypic clusters": row["Heterotypic FITC+ CTC clusters (count/ml)"],
        }
    else:
        if pt not in premortem_data:
            premortem_data[pt] = []

        premortem_data[pt].append(
            {
                "visit": visit,
                "Total CTCs": row["Total FITC+ CTCs (count/ml)"],
                "CTC cluster size": row["Mean CTC cluster size"],
                "Total CTC clusters": row["Total FITC+ CTC clusters (count/ml)"],
                "Single CTCs": row["Single FITC+ CTCs (count/ml)"],
                "Homotypic clusters": row["Homotypic FITC+ CTC clusters (count/ml)"],
                "Heterotypic clusters": row[
                    "Heterotypic FITC+ CTC clusters (count/ml)"
                ],
            }
        )

In [ ]:
x = []
y = []
labs = []

test_vector = []
var = "Total CTCs"

for pt in premortem_data:
    this_ctc_data = [it for it in premortem_data[pt] if not pd.isnull(it[var])]
    this_ctc_data = sorted(this_ctc_data, key=lambda it: it["visit"])

    for idx, data_dict in enumerate(this_ctc_data):
        timepoint = -1 * (len(this_ctc_data) - idx)

        # we are only visualizing timepoints -3 to 0 in these plots, but you will notice that
        # the test vector uses the first available timepoint. In any case, this would only
        # potentially affect a single case and would have no effect on outcome
        if timepoint > -4:
            x.append(timepoint)
            y.append(data_dict[var])
            labs.append(pt)

    postmortem_val = postmortem_data[pt][var]

    if not pd.isnull(postmortem_val):
        x.append(0)
        y.append(postmortem_val)
        labs.append(pt)

        test_vector.append(postmortem_val - this_ctc_data[0][var])

wilcoxon_res = wilcoxon(test_vector, alternative="two-sided")

# pad 0 values for visualization in log scale
y = np.array(y) + 0.05
y = np.log10(y)

pal = sns.color_palette([f"#14140a" for _ in range(len(premortem_data))])

p = sns.lineplot(x=x, y=y, hue=labs, palette=pal, alpha=0.298)

c = sns.scatterplot(x=x, y=y, hue=labs, alpha=0.298, s=60, legend=False, palette=pal)

d = sns.regplot(
    x=x, y=y, scatter=False, ci=None, order=3, line_kws={"linewidth": 5}, ax=plt.gca()
)

_ = plt.yticks(
    [-1, 0, 2, 4],
)

# set y tick labels
_ = p.set_yticklabels(["", "$10^0$", "$10^2$", "$10^4$"])

_ = plt.ylim(-1.5, 4)
_ = plt.xticks([-3, -2, -1, 0])

# remove legend
_ = plt.legend([], [], frameon=False)

_ = plt.xlabel("Pt visits before death")
_ = plt.ylabel("Total CTCs (count/ml)")

ylim = plt.ylim()
counts_y = ylim[0] - (ylim[1] - ylim[0]) * 0.25

_ = plt.text(
    -4.1,
    counts_y,
    "No. Pts",
)
_ = plt.text(
    -3.05,
    counts_y,
    str(len([it for it in x if it == -3])),
)
_ = plt.text(
    -2.1,
    counts_y,
    str(len([it for it in x if it == -2])),
)
_ = plt.text(
    -1.1,
    counts_y,
    str(len([it for it in x if it == -1])),
)
_ = plt.text(
    -0.1,
    counts_y,
    str(len([it for it in x if it == 0])),
)

_ = p.spines["top"].set_visible(False)
_ = p.spines["right"].set_visible(False)

_ = plt.title(f"Wilcoxon p-value: {wilcoxon_res.pvalue:.2e}")

In [ ]:
x = []
y = []
labs = []

test_vector = []
var = "Total CTC clusters"

for pt in premortem_data:
    this_ctc_data = [it for it in premortem_data[pt] if not pd.isnull(it[var])]
    this_ctc_data = sorted(this_ctc_data, key=lambda it: it["visit"])

    for idx, data_dict in enumerate(this_ctc_data):
        timepoint = -1 * (len(this_ctc_data) - idx)

        if timepoint > -4:
            x.append(timepoint)
            y.append(data_dict[var])
            labs.append(pt)

    postmortem_val = postmortem_data[pt][var]

    if not pd.isnull(postmortem_val):
        x.append(0)
        y.append(postmortem_val)
        labs.append(pt)

        test_vector.append(postmortem_val - this_ctc_data[0][var])

wilcoxon_res = wilcoxon(test_vector, alternative="two-sided")

# pad 0 values for visualization in log scale
y = np.array(y) + 0.05
y = np.log10(y)

pal = sns.color_palette([f"#14140a" for _ in range(len(premortem_data))])

p = sns.lineplot(x=x, y=y, hue=labs, palette=pal, alpha=0.298)

c = sns.scatterplot(x=x, y=y, hue=labs, alpha=0.298, s=60, legend=False, palette=pal)

d = sns.regplot(
    x=x, y=y, scatter=False, ci=None, order=3, line_kws={"linewidth": 5}, ax=plt.gca()
)

_ = plt.yticks(
    [-1, 0, 1, 2, 3],
)

# set y tick labels
_ = p.set_yticklabels(["", "$10^0$", "$10^1$", "$10^2$", "$10^3$"])

_ = plt.ylim(-1.5, 3)

_ = plt.xticks([-3, -2, -1, 0])

# remove legend
_ = plt.legend([], [], frameon=False)

_ = plt.xlabel("Pt visits before death")
_ = plt.ylabel("Total CTC clusters (count/ml)")

ylim = plt.ylim()
counts_y = ylim[0] - (ylim[1] - ylim[0]) * 0.25

_ = plt.text(
    -4.1,
    counts_y,
    "No. Pts",
)
_ = plt.text(
    -3.05,
    counts_y,
    str(len([it for it in x if it == -3])),
)
_ = plt.text(
    -2.1,
    counts_y,
    str(len([it for it in x if it == -2])),
)
_ = plt.text(
    -1.1,
    counts_y,
    str(len([it for it in x if it == -1])),
)
_ = plt.text(
    -0.1,
    counts_y,
    str(len([it for it in x if it == 0])),
)

_ = p.spines["top"].set_visible(False)
_ = p.spines["right"].set_visible(False)

_ = plt.title(f"Wilcoxon p-value: {wilcoxon_res.pvalue:.2e}")

In [ ]:
x = []
y = []
labs = []

test_vector = []
var = "Single CTCs"

for pt in premortem_data:
    this_ctc_data = [it for it in premortem_data[pt] if not pd.isnull(it[var])]
    this_ctc_data = sorted(this_ctc_data, key=lambda it: it["visit"])

    for idx, data_dict in enumerate(this_ctc_data):
        timepoint = -1 * (len(this_ctc_data) - idx)

        if timepoint > -4:
            x.append(timepoint)
            y.append(data_dict[var])
            labs.append(pt)

    postmortem_val = postmortem_data[pt][var]

    if not pd.isnull(postmortem_val):
        x.append(0)
        y.append(postmortem_val)
        labs.append(pt)

        test_vector.append(postmortem_val - this_ctc_data[0][var])

wilcoxon_res = wilcoxon(test_vector, alternative="two-sided")

# pad 0 values for visualization in log scale
y = np.array(y) + 0.05
y = np.log10(y)

pal = sns.color_palette([f"#14140a" for _ in range(len(premortem_data))])

p = sns.lineplot(x=x, y=y, hue=labs, palette=pal, alpha=0.298)

c = sns.scatterplot(x=x, y=y, hue=labs, alpha=0.298, s=60, legend=False, palette=pal)

d = sns.regplot(
    x=x, y=y, scatter=False, ci=None, order=3, line_kws={"linewidth": 5}, ax=plt.gca()
)

_ = plt.yticks(
    [-1, 0, 1, 2, 3],
)

# set y tick labels
_ = p.set_yticklabels(["", "$10^0$", "$10^1$", "$10^2$", "$10^3$"])

_ = plt.ylim(-1.5, 3)

_ = plt.xticks([-3, -2, -1, 0])

# remove legend
_ = plt.legend([], [], frameon=False)

_ = plt.xlabel("Pt visits before death")
_ = plt.ylabel("Single CTCs (count/ml)")

ylim = plt.ylim()
counts_y = ylim[0] - (ylim[1] - ylim[0]) * 0.25

_ = plt.text(
    -4.1,
    counts_y,
    "No. Pts",
)
_ = plt.text(
    -3.05,
    counts_y,
    str(len([it for it in x if it == -3])),
)
_ = plt.text(
    -2.1,
    counts_y,
    str(len([it for it in x if it == -2])),
)
_ = plt.text(
    -1.1,
    counts_y,
    str(len([it for it in x if it == -1])),
)
_ = plt.text(
    -0.1,
    counts_y,
    str(len([it for it in x if it == 0])),
)

_ = p.spines["top"].set_visible(False)
_ = p.spines["right"].set_visible(False)

_ = plt.title(f"Wilcoxon p-value: {wilcoxon_res.pvalue:.2e}")

In [ ]:
x = []
y = []
labs = []

test_vector = []
var = "Homotypic clusters"

for pt in premortem_data:
    this_ctc_data = [it for it in premortem_data[pt] if not pd.isnull(it[var])]
    this_ctc_data = sorted(this_ctc_data, key=lambda it: it["visit"])

    for idx, data_dict in enumerate(this_ctc_data):
        timepoint = -1 * (len(this_ctc_data) - idx)

        if timepoint > -4:
            x.append(timepoint)
            y.append(data_dict[var])
            labs.append(pt)

    postmortem_val = postmortem_data[pt][var]

    if not pd.isnull(postmortem_val):
        x.append(0)
        y.append(postmortem_val)
        labs.append(pt)

        test_vector.append(postmortem_val - this_ctc_data[0][var])

wilcoxon_res = wilcoxon(test_vector, alternative="two-sided")

# pad 0 values for visualization in log scale
y = np.array(y) + 0.05
y = np.log10(y)

pal = sns.color_palette([f"#14140a" for _ in range(len(premortem_data))])

p = sns.lineplot(x=x, y=y, hue=labs, palette=pal, alpha=0.298)

c = sns.scatterplot(x=x, y=y, hue=labs, alpha=0.298, s=60, legend=False, palette=pal)

d = sns.regplot(
    x=x, y=y, scatter=False, ci=None, order=3, line_kws={"linewidth": 5}, ax=plt.gca()
)

_ = plt.yticks(
    [
        -1,
        0,
        1,
        2,
    ],
)

# set y tick labels
_ = p.set_yticklabels(
    [
        "",
        "$10^0$",
        "$10^1$",
        "$10^2$",
    ]
)

_ = plt.ylim(-1.5, 2)

_ = plt.xticks([-3, -2, -1, 0])

# remove legend
_ = plt.legend([], [], frameon=False)

_ = plt.xlabel("Pt visits before death")
_ = plt.ylabel(f"{var} (count/ml)")

ylim = plt.ylim()
counts_y = ylim[0] - (ylim[1] - ylim[0]) * 0.25

_ = plt.text(
    -4.1,
    counts_y,
    "No. Pts",
)
_ = plt.text(
    -3.05,
    counts_y,
    str(len([it for it in x if it == -3])),
)
_ = plt.text(
    -2.1,
    counts_y,
    str(len([it for it in x if it == -2])),
)
_ = plt.text(
    -1.1,
    counts_y,
    str(len([it for it in x if it == -1])),
)
_ = plt.text(
    -0.1,
    counts_y,
    str(len([it for it in x if it == 0])),
)

_ = p.spines["top"].set_visible(False)
_ = p.spines["right"].set_visible(False)

_ = plt.title(f"Wilcoxon p-value: {wilcoxon_res.pvalue:.2e}")

In [ ]:
x = []
y = []
labs = []

test_vector = []
var = "Heterotypic clusters"

for pt in premortem_data:
    this_ctc_data = [it for it in premortem_data[pt] if not pd.isnull(it[var])]
    this_ctc_data = sorted(this_ctc_data, key=lambda it: it["visit"])

    for idx, data_dict in enumerate(this_ctc_data):
        timepoint = -1 * (len(this_ctc_data) - idx)

        if timepoint > -4:
            x.append(timepoint)
            y.append(data_dict[var])
            labs.append(pt)

    postmortem_val = postmortem_data[pt][var]

    if not pd.isnull(postmortem_val):
        x.append(0)
        y.append(postmortem_val)
        labs.append(pt)

        test_vector.append(postmortem_val - this_ctc_data[0][var])

wilcoxon_res = wilcoxon(test_vector, alternative="two-sided")

# pad 0 values for visualization in log scale
y = np.array(y) + 0.05
y = np.log10(y)

pal = sns.color_palette([f"#14140a" for _ in range(len(premortem_data))])

p = sns.lineplot(x=x, y=y, hue=labs, palette=pal, alpha=0.298)

c = sns.scatterplot(x=x, y=y, hue=labs, alpha=0.298, s=60, legend=False, palette=pal)

d = sns.regplot(
    x=x, y=y, scatter=False, ci=None, order=3, line_kws={"linewidth": 5}, ax=plt.gca()
)

_ = plt.yticks(
    [-1, 0, 1, 2, 3],
)

# set y tick labels
_ = p.set_yticklabels(["", "$10^0$", "$10^1$", "$10^2$", "$10^3$"])

_ = plt.ylim(-1.5, 3)

_ = plt.xticks([-3, -2, -1, 0])

# remove legend
_ = plt.legend([], [], frameon=False)

_ = plt.xlabel("Pt visits before death")
_ = plt.ylabel(f"{var} (count/ml)")

ylim = plt.ylim()
counts_y = ylim[0] - (ylim[1] - ylim[0]) * 0.25

_ = plt.text(
    -4.1,
    counts_y,
    "No. Pts",
)
_ = plt.text(
    -3.05,
    counts_y,
    str(len([it for it in x if it == -3])),
)
_ = plt.text(
    -2.1,
    counts_y,
    str(len([it for it in x if it == -2])),
)
_ = plt.text(
    -1.1,
    counts_y,
    str(len([it for it in x if it == -1])),
)
_ = plt.text(
    -0.1,
    counts_y,
    str(len([it for it in x if it == 0])),
)

_ = p.spines["top"].set_visible(False)
_ = p.spines["right"].set_visible(False)

_ = plt.title(f"Wilcoxon p-value: {wilcoxon_res.pvalue:.2e}")

In [ ]:
x = []
y = []
labs = []

test_vector = []
var = "CTC cluster size"

for pt in premortem_data:
    this_ctc_data = sorted(premortem_data[pt], key=lambda it: it["visit"])
    premortem_vals = []

    for idx, data_dict in enumerate(this_ctc_data):
        timepoint = -1 * (len(this_ctc_data) - idx)

        if timepoint > -4 and not pd.isnull(data_dict[var]):
            x.append(timepoint)
            y.append(data_dict[var])
            labs.append(pt)

            premortem_vals.append(data_dict[var])

    postmortem_val = postmortem_data[pt][var]

    if not pd.isnull(postmortem_val):
        x.append(0)
        y.append(postmortem_val)
        labs.append(pt)

        if len(this_ctc_data) > 0 and len(premortem_vals) > 0:
            test_vector.append(postmortem_val - premortem_vals[0])

wilcoxon_res = wilcoxon(test_vector, alternative="two-sided")

# pad 0 values for visualization in log scale
y = np.array(y) + 0.05
y = np.log10(y)

pal = sns.color_palette([f"#14140a" for _ in range(len(premortem_data))])

p = sns.lineplot(x=x, y=y, hue=labs, palette=pal, alpha=0.298)

c = sns.scatterplot(x=x, y=y, hue=labs, alpha=0.298, s=60, legend=False, palette=pal)

d = sns.regplot(
    x=x, y=y, scatter=False, ci=None, order=2, line_kws={"linewidth": 5}, ax=plt.gca()
)

_ = plt.yticks(
    [0, 1, 2, 3],
)

# set y tick labels
_ = p.set_yticklabels(
    [
        "$10^0$",
        "$10^1$",
        "$10^2$",
        "$10^3$",
    ]
)

_ = plt.ylim(-0.1, 3)
_ = plt.xticks([-3, -2, -1, 0])

# remove legend
_ = plt.legend([], [], frameon=False)

_ = plt.xlabel("Pt visits before death")
_ = plt.ylabel("CTC cluster size (mean) (count/ml)")

ylim = plt.ylim()
counts_y = ylim[0] - (ylim[1] - ylim[0]) * 0.25

_ = plt.text(
    -4.1,
    counts_y,
    "No. Pts",
)
_ = plt.text(
    -3.05,
    counts_y,
    str(len([it for it in x if it == -3])),
)
_ = plt.text(
    -2.1,
    counts_y,
    str(len([it for it in x if it == -2])),
)
_ = plt.text(
    -1.1,
    counts_y,
    str(len([it for it in x if it == -1])),
)
_ = plt.text(
    -0.1,
    counts_y,
    str(len([it for it in x if it == 0])),
)

_ = p.spines["top"].set_visible(False)
_ = p.spines["right"].set_visible(False)

_ = plt.title(f"Wilcoxon p-value: {wilcoxon_res.pvalue:.3f}")

# German retrospective cohort

## Survival analyses
Logic for the time-varying covariate Cox regression is based on the [`lifelines` documentation](https://lifelines.readthedocs.io/en/latest/Time%20varying%20survival%20regression.html).

In [ ]:
german_df = pd.read_excel("data/Supplementary Table 11.xlsx")

In [ ]:
data = []

# i do this because i don't enjoy pandas
for idx, row in german_df.iterrows():
    vital_status = row["Vital status"]

    if vital_status == "Alive":
        vital_status = 0
    elif vital_status == "Deceased":
        vital_status = 1
    else:
        raise ValueError(f"Unexpected vital status: {vital_status}")

    sex = 0

    if row["Sex"] == "M":
        sex = 1

    vessel_times = []

    for vessel in range(1, 5):
        vessel_col = f"Vessel {vessel} involvement"

        if row[vessel_col] == 1:
            imaging_time_cols = []

            for imaging_idx in range(1, 10):
                this_imaging_time_col = f"Vessel {vessel} Imaging time {imaging_idx} time to involvement (days)"
                if this_imaging_time_col in row:
                    imaging_time_cols.append(this_imaging_time_col)

            final_imaging_time_col = (
                f"Vessel {vessel} LAST AVAILABLE STUDY time to involvement (days)"
            )

            if final_imaging_time_col in row:
                imaging_time_cols.append(final_imaging_time_col)

            involvement_times = [
                row[it] for it in imaging_time_cols if pd.notnull(row[it])
            ]
            earliest_involvement = min(involvement_times)

            vessel_times.append((row[f"Vessel {vessel} type"], earliest_involvement))

    earliest_vessel = None

    if len(vessel_times) > 0:
        min_time = min([it[1] for it in vessel_times])
        earliest_vessel = [it[0] for it in vessel_times if it[1] == min_time]
        earliest_vessel = [str(it) for it in earliest_vessel]

        if len(earliest_vessel) > 1:
            earliest_vessel = " + ".join(sorted(earliest_vessel))
        else:
            earliest_vessel = earliest_vessel[0]

    data.append(
        {
            "pt": row["Censored ID"],
            "age": row["Age"],
            "sex": sex,
            "status": vital_status,
            "cancer_type": row["Cancer Type"],
            "time": row["Time to last followup or death (days)"],
            "delta_to_resection": row["Time to resection (days)"],
            "delta_to_mets": row["Time to first metastatic lesion (days)"],
            "delta_to_involvement": row["Time to vessel involvement (days)"],
            "earliest_vessel": earliest_vessel,
        }
    )

In [ ]:
temp_df = []

for data_dict in data:
    temp_df.append(
        {
            "pt": data_dict["pt"],
            "time": data_dict["time"],
            "status": data_dict["status"],
            "age": data_dict["age"],
            "sex": data_dict["sex"],
        }
    )

temp_df = pd.DataFrame(temp_df)
temp_df = to_long_format(
    temp_df,
    duration_col="time",
)

# add time-varying covariates
for dict_var, cv_var in zip(
    ["delta_to_involvement", "delta_to_resection", "delta_to_mets"],
    ["vessel_involvement", "resection", "mets"],
):
    cv = []

    for data_dict in data:
        if not pd.isnull(data_dict[dict_var]) and data_dict[dict_var] <= 0:
            cv.append({"pt": data_dict["pt"], "time": 0, cv_var: True})
        else:
            cv.append({"pt": data_dict["pt"], "time": 0, cv_var: False})

        if not pd.isnull(data_dict[dict_var]) and data_dict[dict_var] > 0:
            cv.append(
                {"pt": data_dict["pt"], "time": data_dict[dict_var], cv_var: True}
            )

    cv = pd.DataFrame(cv, columns=["pt", "time", cv_var])

    temp_df = add_covariate_to_timeline(
        temp_df, cv, id_col="pt", duration_col="time", event_col="status"
    )

In [ ]:
temp_df = temp_df.dropna()

In [ ]:
mod = CoxTimeVaryingFitter(penalizer=0.1)
_ = mod.fit(temp_df, id_col="pt", event_col="status", show_progress=True)

In [ ]:
mod.summary

In [ ]:
summary_tables = []

for cancer_type in set(german_df["Cancer Type"]):
    temp_df = []

    for data_dict in data:
        if data_dict["cancer_type"] != cancer_type:
            continue

        this_dict = {
            "pt": data_dict["pt"],
            "time": data_dict["time"],
            "status": data_dict["status"],
            "age": data_dict["age"],
            "sex": data_dict["sex"],
        }

        if cancer_type == "Ovarian":
            del this_dict["sex"]

        temp_df.append(this_dict)

    temp_df = pd.DataFrame(temp_df)
    temp_df = to_long_format(
        temp_df,
        duration_col="time",
    )

    # add time-varying covariates
    for dict_var, cv_var in zip(
        ["delta_to_involvement", "delta_to_resection", "delta_to_mets"],
        ["vessel_involvement", "resection", "mets"],
    ):
        cv = []

        for data_dict in data:
            if not pd.isnull(data_dict[dict_var]) and data_dict[dict_var] <= 0:
                cv.append({"pt": data_dict["pt"], "time": 0, cv_var: True})
            else:
                cv.append({"pt": data_dict["pt"], "time": 0, cv_var: False})

            if not pd.isnull(data_dict[dict_var]) and data_dict[dict_var] > 0:
                cv.append(
                    {"pt": data_dict["pt"], "time": data_dict[dict_var], cv_var: True}
                )

        cv = pd.DataFrame(cv, columns=["pt", "time", cv_var])

        temp_df = add_covariate_to_timeline(
            temp_df, cv, id_col="pt", duration_col="time", event_col="status"
        )

    temp_df = temp_df.dropna()

    mod = CoxTimeVaryingFitter(penalizer=0.1)
    _ = mod.fit(temp_df, id_col="pt", event_col="status", show_progress=False)

    summary_tables.append({"cancer_type": cancer_type, "summary": mod.summary})

In [ ]:
for summary_table in summary_tables:
    print(f"###############################\n\n" f"{summary_table['cancer_type']}")
    display(summary_table["summary"])
    print("\n\n")

In [ ]:
temp_df = []

for data_dict in data:
    if pd.isnull(data_dict["delta_to_mets"]):
        continue

    group = "$V_0$"

    if (
        not pd.isnull(data_dict["delta_to_involvement"])
        and data_dict["delta_to_involvement"] <= data_dict["delta_to_mets"] + 30
    ):
        group = "$V_1$"

    temp_df.append(
        {
            "time": (
                data_dict["time"] - data_dict["delta_to_mets"]
            ),  # time from metastasis
            "status": data_dict["status"],
            "group": group,
        }
    )

temp_df = pd.DataFrame(temp_df)

# Yes, on average, years are 365.2524 days. We use 365 here
temp_df["time"] = temp_df["time"] / 365

In [ ]:
ax = plt.subplot(111)

kmf_0 = lifelines.KaplanMeierFitter()
kmf_0.fit(
    temp_df.loc[temp_df["group"] == "$V_0$"]["time"],
    event_observed=temp_df.loc[temp_df["group"] == "$V_0$"]["status"],
    label=f"$V_0$, n={temp_df.loc[temp_df['group'] == '$V_0$'].shape[0]}",
)
kmf_0.plot(
    ax=ax,
    ci_show=False,
    show_censors=True,
    linewidth=2.3,
)

kmf_1 = lifelines.KaplanMeierFitter()
kmf_1.fit(
    temp_df.loc[temp_df["group"] == "$V_1$"]["time"],
    event_observed=temp_df.loc[temp_df["group"] == "$V_1$"]["status"],
    label=f"$V_1$, n={temp_df.loc[temp_df['group'] == '$V_1$'].shape[0]}",
)
kmf_1.plot(ax=ax, ci_show=False, show_censors=True, linewidth=2.3)

_ = plt.xlim(0, 5)
# add at risk counts
lifelines.plotting.add_at_risk_counts(kmf_0, kmf_1, ax=ax)

res = lifelines.statistics.logrank_test(
    temp_df.loc[temp_df["group"] == "$V_0$"]["time"],
    temp_df.loc[temp_df["group"] == "$V_1$"]["time"],
    event_observed_A=temp_df.loc[temp_df["group"] == "$V_0$"]["status"],
    event_observed_B=temp_df.loc[temp_df["group"] == "$V_1$"]["status"],
)

_ = plt.xlabel("Years")
_ = plt.ylabel("Survival Probability")

_ = plt.title(
    f"Survival from time of metastasis\np={res.p_value: 0.3e}\n"
    f"Median survival: {round(kmf_0.median_survival_time_, 2)} vs. "
    f"{round(kmf_1.median_survival_time_, 2)} years"
)

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(5.22 * 5, 6.5))

for idx, cancer_type in enumerate(german_df["Cancer Type"].unique()):
    temp_df = []

    for data_dict in data:
        if (
            pd.isnull(data_dict["delta_to_mets"])
            or data_dict["cancer_type"] != cancer_type
        ):
            continue

        group = "$V_0$"

        if (
            not pd.isnull(data_dict["delta_to_involvement"])
            and data_dict["delta_to_involvement"] <= data_dict["delta_to_mets"] + 30
        ):
            group = "$V_1$"

        temp_df.append(
            {
                "time": (data_dict["time"] - data_dict["delta_to_mets"]),
                "status": data_dict["status"],
                "group": group,
            }
        )

    temp_df = pd.DataFrame(temp_df)

    temp_df["time"] = temp_df["time"] / 365

    kmf_0 = lifelines.KaplanMeierFitter()
    kmf_0.fit(
        temp_df.loc[temp_df["group"] == "$V_0$"]["time"],
        event_observed=temp_df.loc[temp_df["group"] == "$V_0$"]["status"],
        label=f"$V_0$, n={temp_df.loc[temp_df['group'] == '$V_0$'].shape[0]}",
    )
    kmf_0.plot(
        ax=ax[idx],
        ci_show=False,
        show_censors=True,
        linewidth=2.3,
    )

    kmf_1 = lifelines.KaplanMeierFitter()
    kmf_1.fit(
        temp_df.loc[temp_df["group"] == "$V_1$"]["time"],
        event_observed=temp_df.loc[temp_df["group"] == "$V_1$"]["status"],
        label=f"$V_1$, n={temp_df.loc[temp_df['group'] == '$V_1$'].shape[0]}",
    )
    kmf_1.plot(ax=ax[idx], ci_show=False, show_censors=True, linewidth=2.3)

    _ = plt.xlim(0, 5)
    # add at risk counts
    lifelines.plotting.add_at_risk_counts(
        kmf_0,
        kmf_1,
        ax=ax[idx],
        xticks=[0, 1, 2, 3, 4, 5],
        rows_to_show=["At risk", "Censored", "Events"],
    )

    res = lifelines.statistics.logrank_test(
        temp_df.loc[temp_df["group"] == "$V_0$"]["time"],
        temp_df.loc[temp_df["group"] == "$V_1$"]["time"],
        event_observed_A=temp_df.loc[temp_df["group"] == "$V_0$"]["status"],
        event_observed_B=temp_df.loc[temp_df["group"] == "$V_1$"]["status"],
    )

    _ = ax[idx].set_xlabel("Years")
    _ = ax[idx].set_ylabel("Survival Probability")

    _ = ax[idx].set_title(
        f"{cancer_type}\nSurvival from time of metastasis\np={res.p_value: 0.3e}\n"
        f"Median survival: {round(kmf_0.median_survival_time_, 2)} vs. "
        f"{round(kmf_1.median_survival_time_, 2)} years"
    )

    _ = ax[idx].set_xlim((0, 5))

_ = fig.tight_layout()

In [ ]:
counts = {}

for it in data:
    if it["earliest_vessel"] not in counts:
        counts[it["earliest_vessel"]] = 0

    counts[it["earliest_vessel"]] += 1

counts = [(k, v) for k, v in counts.items()]
counts = sorted(counts, key=lambda x: x[1], reverse=True)
top_vessels = [it[0] for it in counts[1:6]]

In [ ]:
counts[:6]

In [ ]:
temp_df = []

for data_dict in data:
    temp_df.append(
        {
            "pt": data_dict["pt"],
            "time": data_dict["time"],
            "status": data_dict["status"],
        }
    )

temp_df = pd.DataFrame(temp_df)
temp_df = to_long_format(
    temp_df,
    duration_col="time",
)

################## vessel involvement
cv = []

for data_dict in data:
    if (
        not pd.isnull(data_dict["delta_to_involvement"])
        and data_dict["delta_to_involvement"] <= 0
    ):
        this_vessel = "Other"

        if data_dict["earliest_vessel"] in top_vessels:
            this_vessel = f"{data_dict['earliest_vessel']} only"

        cv.append({"pt": data_dict["pt"], "time": 0, "vessel_involved": this_vessel})
    else:
        cv.append({"pt": data_dict["pt"], "time": 0, "vessel_involved": "None"})

    if (
        not pd.isnull(data_dict["delta_to_involvement"])
        and data_dict["delta_to_involvement"] > 0
    ):
        this_vessel = "Other"

        if data_dict["earliest_vessel"] in top_vessels:
            this_vessel = f"{data_dict['earliest_vessel']} only"

        cv.append(
            {
                "pt": data_dict["pt"],
                "time": data_dict["delta_to_involvement"],
                "vessel_involved": this_vessel,
            }
        )

cv = pd.DataFrame(cv, columns=["pt", "time", "vessel_involved"])

temp_df = add_covariate_to_timeline(
    temp_df, cv, id_col="pt", duration_col="time", event_col="status"
)

In [ ]:
temp_df = pd.get_dummies(temp_df, columns=["vessel_involved"], drop_first=False)
temp_df = temp_df.drop(columns=["vessel_involved_None"])

# drop any rows with NaNs from dataset
temp_df = temp_df.dropna()

temp_df = temp_df.loc[
    ~((temp_df["start"] == temp_df["stop"]) & (temp_df["start"] == 0))
]

mod = CoxTimeVaryingFitter(penalizer=0.1)
_ = mod.fit(temp_df, id_col="pt", event_col="status", show_progress=True)

In [ ]:
mod.summary

In [ ]:
temp_df = []

for data_dict in data:
    temp_df.append(
        {
            "pt": data_dict["pt"],
            "time": data_dict["time"],
            "status": data_dict["status"],
            "age": data_dict["age"],
            "sex": data_dict["sex"],
        }
    )

temp_df = pd.DataFrame(temp_df)
temp_df = to_long_format(
    temp_df,
    duration_col="time",
)

################## vessel involvement
cv = []

for data_dict in data:
    if (
        not pd.isnull(data_dict["delta_to_involvement"])
        and data_dict["delta_to_involvement"] <= 0
    ):
        this_vessel = "Other"

        if data_dict["earliest_vessel"] in top_vessels:
            this_vessel = f"{data_dict['earliest_vessel']} only"

        cv.append({"pt": data_dict["pt"], "time": 0, "vessel_involved": this_vessel})
    else:
        cv.append({"pt": data_dict["pt"], "time": 0, "vessel_involved": "None"})

    if (
        not pd.isnull(data_dict["delta_to_involvement"])
        and data_dict["delta_to_involvement"] > 0
    ):
        this_vessel = "Other"

        if data_dict["earliest_vessel"] in top_vessels:
            this_vessel = f"{data_dict['earliest_vessel']} only"

        cv.append(
            {
                "pt": data_dict["pt"],
                "time": data_dict["delta_to_involvement"],
                "vessel_involved": this_vessel,
            }
        )

cv = pd.DataFrame(cv, columns=["pt", "time", "vessel_involved"])

temp_df = add_covariate_to_timeline(
    temp_df, cv, id_col="pt", duration_col="time", event_col="status"
)

#################### resection and mets
for dict_var, cv_var in zip(
    ["delta_to_resection", "delta_to_mets"], ["resection", "mets"]
):
    cv = []

    for data_dict in data:
        if not pd.isnull(data_dict[dict_var]) and data_dict[dict_var] <= 0:
            cv.append({"pt": data_dict["pt"], "time": 0, cv_var: True})
        else:
            cv.append({"pt": data_dict["pt"], "time": 0, cv_var: False})

        if not pd.isnull(data_dict[dict_var]) and data_dict[dict_var] > 0:
            cv.append(
                {"pt": data_dict["pt"], "time": data_dict[dict_var], cv_var: True}
            )

    cv = pd.DataFrame(cv, columns=["pt", "time", cv_var])

    temp_df = add_covariate_to_timeline(
        temp_df, cv, id_col="pt", duration_col="time", event_col="status"
    )

In [ ]:
temp_df = pd.get_dummies(temp_df, columns=["vessel_involved"], drop_first=False)
temp_df = temp_df.drop(columns=["vessel_involved_None"])

In [ ]:
# drop any rows with NaNs from dataset
temp_df = temp_df.dropna()

mod = CoxTimeVaryingFitter(penalizer=0.1)
_ = mod.fit(temp_df, id_col="pt", event_col="status", show_progress=True)

In [ ]:
mod.summary

## individual cancer types

In [ ]:
for cancer_type in set(german_df["Cancer Type"]):
    counts = {}

    for it in [pt for pt in data if pt["cancer_type"] == cancer_type]:

        if it["earliest_vessel"] not in counts:
            counts[it["earliest_vessel"]] = 0

        counts[it["earliest_vessel"]] += 1

    counts = [(k, v) for k, v in counts.items()]
    counts = sorted(counts, key=lambda x: x[1], reverse=True)
    top_vessels = [it[0] for it in counts[1:] if it[1] >= 10]

    print(
        f"###############################\n\n"
        f"{cancer_type}\n"
        f"Top vessels: {', '.join(top_vessels)}\n"
    )

    for k, v in counts[:6]:
        print(f"{k}: {v}")

    temp_df = []

    for data_dict in [pt for pt in data if pt["cancer_type"] == cancer_type]:

        this_pt_dict = {
            "pt": data_dict["pt"],
            "time": data_dict["time"],
            "status": data_dict["status"],
        }

        temp_df.append(this_pt_dict)

    temp_df = pd.DataFrame(temp_df)
    temp_df = to_long_format(
        temp_df,
        duration_col="time",
    )

    ################## vessel involvement
    cv = []

    for data_dict in [pt for pt in data if pt["cancer_type"] == cancer_type]:
        if (
            not pd.isnull(data_dict["delta_to_involvement"])
            and data_dict["delta_to_involvement"] <= 0
        ):
            this_vessel = "Other"

            if data_dict["earliest_vessel"] in top_vessels:
                this_vessel = f"{data_dict['earliest_vessel']} only"

            cv.append(
                {"pt": data_dict["pt"], "time": 0, "vessel_involved": this_vessel}
            )
        else:
            cv.append({"pt": data_dict["pt"], "time": 0, "vessel_involved": "None"})

        if (
            not pd.isnull(data_dict["delta_to_involvement"])
            and data_dict["delta_to_involvement"] > 0
        ):
            this_vessel = "Other"

            if data_dict["earliest_vessel"] in top_vessels:
                this_vessel = f"{data_dict['earliest_vessel']} only"

            cv.append(
                {
                    "pt": data_dict["pt"],
                    "time": data_dict["delta_to_involvement"],
                    "vessel_involved": this_vessel,
                }
            )

    cv = pd.DataFrame(cv, columns=["pt", "time", "vessel_involved"])

    temp_df = add_covariate_to_timeline(
        temp_df, cv, id_col="pt", duration_col="time", event_col="status"
    )

    #################### resection and mets
    for dict_var, cv_var in zip(
        ["delta_to_resection", "delta_to_mets"], ["resection", "mets"]
    ):
        cv = []

        for data_dict in [pt for pt in data if pt["cancer_type"] == cancer_type]:
            if not pd.isnull(data_dict[dict_var]) and data_dict[dict_var] <= 0:
                cv.append({"pt": data_dict["pt"], "time": 0, cv_var: True})
            else:
                cv.append({"pt": data_dict["pt"], "time": 0, cv_var: False})

            if not pd.isnull(data_dict[dict_var]) and data_dict[dict_var] > 0:
                cv.append(
                    {"pt": data_dict["pt"], "time": data_dict[dict_var], cv_var: True}
                )

        cv = pd.DataFrame(cv, columns=["pt", "time", cv_var])

        temp_df = add_covariate_to_timeline(
            temp_df, cv, id_col="pt", duration_col="time", event_col="status"
        )

    temp_df = pd.get_dummies(temp_df, columns=["vessel_involved"], drop_first=False)
    temp_df = temp_df.drop(columns=["vessel_involved_None"])
    # drop any rows with NaNs from dataset
    temp_df = temp_df.dropna()

    mod = CoxTimeVaryingFitter(penalizer=0.1)
    _ = mod.fit(temp_df, id_col="pt", event_col="status", show_progress=False)

    display(mod.summary)

    print(len(temp_df["pt"].unique()), "patients in this cancer type")

## Frequencies of vessel involvement by cancer type

In [ ]:
counts = {}

for idx, row in german_df.iterrows():
    cancer_type = row["Cancer Type"]

    vessels = []

    for vessel_idx in range(1, 5):
        vessel_col = f"Vessel {vessel_idx} involvement"
        if pd.notna(row[vessel_col]) and row[vessel_col] == 1:
            if pd.isnull(row[f"Vessel {vessel_idx} type"]):
                continue

            vessels.append(row[f"Vessel {vessel_idx} type"])

    for vessel in vessels:
        id = (cancer_type, vessel)

        if id not in counts:
            counts[id] = 0

        counts[id] += 1

In [ ]:
unique_cancer_types = list(set([it[0] for it in counts]))
unique_vessels = list(set([it[1] for it in counts]))

heatmap = np.zeros((len(unique_cancer_types), len(unique_vessels)), dtype=int)

for (cancer_type, vessel), count in counts.items():
    cancer_type_idx = unique_cancer_types.index(cancer_type)
    vessel_idx = unique_vessels.index(vessel)
    heatmap[cancer_type_idx, vessel_idx] = count

In [ ]:
p = sns.clustermap(
    heatmap.T,
    cmap="vlag",
    center=0,
    figsize=(6, 6),
    yticklabels=unique_vessels,
    xticklabels=unique_cancer_types,
)

## Time to detection


In [ ]:
vals = []
labs = []

for _idx, row in german_df.iterrows():
    vessel_times = []

    for vessel in range(1, 5):
        vessel_col = f"Vessel {vessel} involvement"

        if row[vessel_col] == 1:
            imaging_time_cols = []

            for imaging_idx in range(1, 10):
                this_imaging_time_col = f"Vessel {vessel} Imaging time {imaging_idx} time to involvement (days)"
                if this_imaging_time_col in row:
                    imaging_time_cols.append(this_imaging_time_col)

            final_imaging_time_col = (
                f"Vessel {vessel} LAST AVAILABLE STUDY time to involvement (days)"
            )

            if final_imaging_time_col in row:
                imaging_time_cols.append(final_imaging_time_col)

            involvement_times = [
                row[it] for it in imaging_time_cols if pd.notnull(row[it])
            ]
            earliest_involvement = min(involvement_times)

            vessel_times.append((row[f"Vessel {vessel} type"], earliest_involvement))

    earliest_vessel = None

    if len(vessel_times) > 0:
        min_time = min([it[1] for it in vessel_times])

        vals.append(min_time / 365.25)
        labs.append(row["Cancer Type"])

all_cancer_types = list(set(labs))

dists = [[] for _ in all_cancer_types]

for val, lab in zip(vals, labs):
    dists[all_cancer_types.index(lab)].append(val)

kruskal_result = kruskal(*dists)

In [ ]:
p = sns.boxplot(
    x=labs, y=vals, saturation=0.8, color="white", fliersize=0, **boxplot_props
)
b = sns.stripplot(
    x=labs,
    y=vals,
    alpha=0.7,
    size=6,
)
_ = plt.ylabel("Time to $V_1$ detection (yrs)")

_ = plt.title(
    f"Time to detection by cancer type\nKruskal-Wallis p={kruskal_result.pvalue:.3e}"
)

_ = plt.ylim((-0.5, 20))
# make yticks linspace
yticks = np.linspace(0, 20, 5)
_ = plt.yticks(yticks)

xticklabels = plt.xticks()[1]
xticklabels = [it.get_text() for it in xticklabels]

# set xticks
x_tick_coords = []
x_tick_labs = []

groups = all_cancer_types

for idx, group in enumerate(xticklabels):
    group_split = "\n".join(group.split())
    x_tick_coords.append(idx)
    x_tick_labs.append(
        f"{group_split}\nn={len([v for idx, v in enumerate(vals) if labs[idx] == group])}"
    )

_ = plt.xticks(x_tick_coords, x_tick_labs)
sns.despine()